In [1]:
import paho.mqtt.client as mqtt
import uuid
import platform
import psutil
import time
import json
import redis
from redis.commands.json.path import Path

#PERSONAL PARAMETERS YOU CAN OBTAIN FROM YOUR REDIS ACCOUNT
REDIS_HOST = "## ADD YOU REDIS HOST HERE ##"
REDIS_PORT = 00000   #ADD YOUR REDIS PORT HERE
REDIS_USERNAME = "## ADD YOUR REDIS USERNAME HERE ##"
REDIS_PASSWORD = "## ADD YOUR REDIS PASSWORD HERE ##"

#ADD HERE THE TOPIC YOU WANT TO SUBSCRIBE TO
TOPIC = "## ADD TOPIC HERE ##"

# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print(f'Redis is connected:  {is_connected}')


Redis is connected:  True


In [2]:
def safe_ts_create(key):
    try:
        # to create a redis time series
        redis_client.delete(key)
        redis_client.ts().create(key)
    except redis.ResponseError:
        pass

In [3]:
# Set up MQTT client and callbacks
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+ str(rc))
    client.subscribe(TOPIC)

def on_message(client, userdata, msg):
    #Extracting all the monitored macs
    keys = redis_client.keys('macs:*') #sort of regex 
    monitored_macs = []
    for key in keys:
        key = key.decode()
        monitored_macs.append(key)
    print('monitored macs')
    print(monitored_macs)
    
    message = msg.payload.decode() #returns a string with all the info needed
    print(f"Received message '{message}' on topic {msg.topic}")
    #print(type(msg))
    #print(type(message))
    data = json.loads(message)

    # Extract MAC address and data from message
    mac_address = data['mac_address']
    mac_battery_name = mac_address + ':battery'
    mac_power_name = mac_address + ':power'

    if f'macs:{mac_address}' not in monitored_macs:
        # Create the two time series for each MAC addres: battery, power
        redis_client.json().set(f'macs:{mac_address}', Path.root_path(), mac_address)
        safe_ts_create(mac_battery_name)
        safe_ts_create(mac_power_name)
        print('created new timeseries and also added the new mac to the list of monitored macs')

    #Extract timestamp
    timestamp = data['timestamp']
    print(timestamp)

    #Extract battery level
    battery_level = data['battery_level']
    print(battery_level)

    #Extract power plugged
    power_plugged = data['power_plugged']
    print(power_plugged)

    redis_client.ts().add(mac_battery_name, timestamp, battery_level)
    redis_client.ts().add(mac_power_name, timestamp, power_plugged)        
    print('Added on Redis')
  


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# Connect to MQTT broker
client.connect('mqtt.eclipseprojects.io', 1883) #The MQTT broker can be changed

# Start MQTT event loop
client.loop_forever()


Connected with result code 0
monitored macs
['macs:0x8b84a2a23fd8', 'macs:0xd05099a8ae93', 'macs:0xd1b389213477']
Received message '{"mac_address": "0xd1b389213477", "timestamp": 1673687368611, "battery_level": 37, "power_plugged": 1}' on topic s303452
1673687368611
37
1
Added on Redis
monitored macs
['macs:0x8b84a2a23fd8', 'macs:0xd05099a8ae93', 'macs:0xd1b389213477']
Received message '{"mac_address": "0xd05099a8ae93", "timestamp": 1673687370788, "battery_level": 10, "power_plugged": 1}' on topic s303452
1673687370788
10
1
Added on Redis
monitored macs
['macs:0x8b84a2a23fd8', 'macs:0xd05099a8ae93', 'macs:0xd1b389213477']
Received message '{"mac_address": "0xd1b389213477", "timestamp": 1673687369611, "battery_level": 37, "power_plugged": 1}' on topic s303452
1673687369611
37
1
Added on Redis
monitored macs
['macs:0x8b84a2a23fd8', 'macs:0xd05099a8ae93', 'macs:0xd1b389213477']
Received message '{"mac_address": "0xd05099a8ae93", "timestamp": 1673687371790, "battery_level": 10, "power_plug

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6729a98-a1be-4c88-8020-81eb03e99821' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>